In [ ]:
from labda import Subject
from pathlib import Path

from labda.accelerometer import WEAR_ALGORITHMS, COUNTS_CUT_POINTS
from labda.spatial import TRANSPORTATION_CUT_POINTS


folder = Path("../temp/ipen_denmark/Processed")
id = 560102

gps = Subject.from_parquet(folder / f"GPS/{id}.parquet")
acc = Subject.from_parquet(folder / f"Accelerometers/{id}.parquet")

gps.merge(acc, how="outer")
sbj = gps
del acc, gps

sbj.add_vector_magnitude("counts", name="vm_counts", overwrite=True)
sbj.detect_wear(WEAR_ALGORITHMS["choi_2011"], overwrite=True)

wear_times, days = sbj.get_wear_times(duration="10h", drop="acc+gnss")
sbj.detect_activity_intensity(COUNTS_CUT_POINTS["evenson_children_2018"])

sbj.detect_indoor(overwrite=True)
sbj.detect_trips(overwrite=True)
sbj.detect_transportation(
    TRANSPORTATION_CUT_POINTS["heidler_intensity_2025"], overwrite=True
)

sbj.add_speed(overwrite=True)
sbj.add_distance(overwrite=True)

sbj.to_parquet(folder.parent / "testing.parquet", overwrite=True)

In [ ]:
from labda import Subject
from pathlib import Path

folder = Path("../temp/ipen_denmark/Processed")
sbj = Subject.from_parquet(folder.parent / "testing.parquet")

# sbj.plot("map", color="trip_transport")
# sbj.get_timeline("transport")

In [ ]:
from typing import Any

import pydeck as pdk
from shapely.geometry import Point


def get_map(center: Point, layers: list[pdk.Layer]) -> Any:
    view_state = pdk.ViewState(longitude=center.x, latitude=center.y, zoom=10)

    map = pdk.Deck(
        layers=layers,
        initial_view_state=view_state,
        tooltip={"html": "{tooltip}"},  # type: ignore
        # map_style=pdk.map_styles.CARTO_LIGHT,
    )

    return map

In [ ]:
import networkx as nx
import osmnx as ox
from PIL import ImageColor
import pydeck as pdk
from labda.spatial.utils import df_to_gdf
import pandas as pd

df = sbj.df.copy()

trip = df.loc[df["trip_id"] == 22]

trip = df_to_gdf(trip, crs=sbj.metadata.crs)
trip.to_crs("EPSG:4326", inplace=True)

layer = pdk.Layer(
    "GeoJsonLayer",
    trip,
    stroked=True,
    filled=True,
    get_position="geometry.coordinates",
    get_fill_color=ImageColor.getcolor("red", "RGB"),
    get_line_width=5,
    get_radius=10,
    # get_line_color="color",
    pickable=True,
    auto_highlight=True,
    # highlight_color=ImageColor.getcolor("red", "RGB"),
    opacity=1,
)

layers = [layer]
center = trip["geometry"].union_all().centroid

map = get_map(center, layers)
map

In [57]:
from mappymatch import package_root
from mappymatch.constructs.geofence import Geofence
from mappymatch.constructs.trace import Trace
from mappymatch.maps.nx.nx_map import NxMap
from mappymatch.matchers.lcss.lcss import LCSSMatcher

polygon = trip.union_all().convex_hull.buffer(0.001)
G = ox.graph_from_polygon(polygon, network_type="drive")  # type: ignore

trace = Trace.from_geo_dataframe(trip)
geofence = Geofence.from_trace(trace, padding=1e3)
nx_map = NxMap.from_geofence(geofence)
matcher = LCSSMatcher(nx_map)
matches = matcher.match_trace(trace)

df = matches.matches_to_dataframe()
gdf = matches.matches_to_geodataframe()
gdf.to_crs("EPSG:4326", inplace=True)
gdf.rename(columns={"geometry": "geom"}, inplace=True)

layer = pdk.Layer(
    "GeoJsonLayer",
    gdf,
    stroked=True,
    filled=True,
    # get_position="geometry",
    get_fill_color=ImageColor.getcolor("green", "RGB"),
    get_line_width=5,
    get_radius=10,
    get_line_color=ImageColor.getcolor("green", "RGB"),
    pickable=True,
    auto_highlight=True,
    # highlight_color=ImageColor.getcolor("red", "RGB"),
    opacity=1,
)

layers = [layer]
# center = gdf["geometry"].union_all().centroid


map = get_map(center, layers)

{'osmid': 824803301, 'highway': 'tertiary', 'lanes': '2', 'maxspeed': '50', 'name': 'Bogensevej', 'oneway': False, 'reversed': False, 'length': 18.5214360245491, 'speed_kph': 50.0, 'travel_time': 1.3335433937675354, 'kilometers': 0.0185214360245491}
{'osmid': 372439609, 'highway': 'tertiary', 'lanes': '2', 'maxspeed': '50', 'name': 'Bogensevej', 'oneway': False, 'reversed': True, 'length': 89.52957522891667, 'speed_kph': 50.0, 'travel_time': 6.446129416482, 'kilometers': 0.08952957522891668}
{'osmid': 11559425, 'highway': 'residential', 'maxspeed': '50', 'name': 'Banevænget', 'oneway': False, 'reversed': True, 'length': 41.833864526692054, 'speed_kph': 50.0, 'travel_time': 3.0120382459218282, 'kilometers': 0.041833864526692056}
{'osmid': 11560085, 'highway': 'residential', 'name': 'Bøgeløkken', 'oneway': False, 'reversed': False, 'length': 48.286264125686316, 'speed_kph': 50.0, 'travel_time': 3.476611017049415, 'kilometers': 0.04828626412568632}
{'osmid': 11560085, 'highway': 'resident

In [ ]:
polygon = trip.union_all().convex_hull.buffer(0.001)
G = ox.graph_from_polygon(polygon, network_type="drive")  # type: ignore

fig, ax = ox.plot.plot_graph(
    G,
    show=False,
    close=False,
    bgcolor="#111111",
    edge_color="#ffcb00",
    edge_linewidth=0.3,
    node_size=0,
)

# to this matplotlib axis, add the place shape(s)
trip.plot(ax=ax, fc="#444444", ec=None, lw=1, alpha=1, zorder=-1)

# optionally set up the axes extents
margin = 0.02
west, south, east, north = gdf.union_all().bounds
margin_ns = (north - south) * margin
margin_ew = (east - west) * margin
ax.set_ylim((south - margin_ns, north + margin_ns))
ax.set_xlim((west - margin_ew, east + margin_ew))
plt.show()